<a href="https://colab.research.google.com/github/wayne0git/ml_cv_basics/blob/master/object_detection/yolo_v1_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO v1 Example
### https://pyimagesearch.com/2022/04/11/understanding-a-real-time-object-detection-network-you-only-look-once-yolov1/

## Environment Preparation
## (Assume CUDA, CUDNN, and NVIDIA Driver Installed)

In [1]:
# Sanity check for GPU as runtime
!nvidia-smi

Sat May 14 11:52:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install OpenCV, ffmpeg modules
!apt install libopencv-dev python-opencv ffmpeg

In [ ]:
# Clone AlexeyAB darknet repository
!git clone https://github.com/AlexeyAB/darknet/

In [ ]:
# Change the directory to darknet 
%cd darknet

In [ ]:
# Enable the OpenCV, CUDA, CUDNN, CUDNN_HALF & LIBSO Flags and Compile Darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/g' Makefile
!make

In [ ]:
# Download YOLOv1 Weights
!wget http://pjreddie.com/media/files/yolov1/yolov1.weights

In [ ]:
# Download YOLOv2 Weights
!wget http://pjreddie.com/media/files/yolov2-voc.weights

# Inference

In [ ]:
# Run the darknet image inference script
# --input : Path to the images directory or text file with the path to the images or a single image name
# --weights : YOLOv1 weights path.
# --config_file : Configuration file path for YOLOv1.
# --data_file : Here, we pass the Pascal VOC labels file.
# --dont_show : This will disable OpenCV from displaying the inference results, and we use this since we are working with colab.
!python3 darknet_images.py --input data --weights yolov1.weights --config_file cfg/yolov1/yolo.cfg --data_file cfg/voc.data --dont_show

In [ ]:
# YOLOv2
!python3 darknet_images.py --input data --weights yolov2-voc.weights --config_file cfg/yolov2-voc.cfg --data_file cfg/voc.data --dont_show